In [2]:
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [3]:
# 1️⃣ 데이터 로드
df = pd.read_csv("./data/train.csv")

# 2️⃣ 최근 10년(2015년 이후) 구간 추정
# → 1년에 약 252 거래일, 즉 10년 ≈ 2520일
# 최신 date_id 확인
max_date = df['date_id'].max()
recent_start = max_date - 2520

# 2015년 이후 데이터 필터링
df_recent = df[df['date_id'] >= recent_start].reset_index(drop=True)

print(f"원본 데이터 크기: {df.shape}")
print(f"최근 10년 데이터 크기: {df_recent.shape}")
print(f"date_id 범위: {df_recent['date_id'].min()} ~ {df_recent['date_id'].max()}")

# 3️⃣ 불필요한 컬럼 제거
TARGET = "forward_returns"
DROP_COLS = [TARGET, "risk_free_rate", "market_forward_excess_returns"]

X = df_recent.drop(columns=DROP_COLS).copy()
y = df_recent[TARGET].copy()

# 4️⃣ 결측치 처리
# 금융 피처는 0으로 대체보다 중앙값이 안전함 (극단치 방지)
X = X.fillna(X.median())

# 5️⃣ 확인
print("\n결측치 비율 상위 5개:")
print(X.isnull().mean().sort_values(ascending=False).head())
print(f"\n최종 Feature 수: {X.shape[1]}")
print(f"Target 샘플 수: {y.shape[0]}")


원본 데이터 크기: (8990, 98)
최근 10년 데이터 크기: (2521, 98)
date_id 범위: 6469 ~ 8989

결측치 비율 상위 5개:
date_id    0.0
P12        0.0
P9         0.0
P8         0.0
P7         0.0
dtype: float64

최종 Feature 수: 95
Target 샘플 수: 2521
